In [100]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 253)

In [101]:
unimelb = pd.read_csv('./data/unimelb_training.csv')

/home/charles/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (90,105,119,120,123,124,132,134,135,138,139,147,149,150,153,154,162,164,165,168,169,177,179,183,184,192,194,198,199,207,209,213,214,224,237,239,244) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [113]:
unimelb.shape

(8708, 252)

In [103]:
col_names = list(unimelb.columns.values)
feature_names = list(unimelb.columns.values)
target_names = col_names.pop(1)

In [104]:
unimelb.isnull().any()

Grant.Application.ID                       False
Grant.Status                               False
Sponsor.Code                                True
Grant.Category.Code                         True
Contract.Value.Band...see.note.A            True
Start.date                                 False
RFCD.Code.1                                 True
RFCD.Percentage.1                           True
RFCD.Code.2                                 True
RFCD.Percentage.2                           True
RFCD.Code.3                                 True
RFCD.Percentage.3                           True
RFCD.Code.4                                 True
RFCD.Percentage.4                           True
RFCD.Code.5                                 True
RFCD.Percentage.5                           True
SEO.Code.1                                  True
SEO.Percentage.1                            True
SEO.Code.2                                  True
SEO.Percentage.2                            True
SEO.Code.3          

In [105]:
(unimelb.groupby("Person.ID.1").count()).shape # Person.ID.1 is unique

(2301, 251)

In [106]:
any((item.startswith('Person.ID.15') for item in col_names)) # Other person IDs till 15

True

In [107]:
pd.unique(unimelb["Role.1"])

array(['CHIEF_INVESTIGATOR', 'PRINCIPAL_SUPERVISOR',
       'DELEGATED_RESEARCHER', 'EXT_CHIEF_INVESTIGATOR', nan,
       'STUD_CHIEF_INVESTIGATOR', 'HONVISIT', 'EXTERNAL_ADVISOR'], dtype=object)

In [108]:
pd.unique(unimelb["Role.2"])

array([nan, 'CHIEF_INVESTIGATOR', 'EXT_CHIEF_INVESTIGATOR',
       'STUD_CHIEF_INVESTIGATOR', 'DELEGATED_RESEARCHER',
       'PRINCIPAL_SUPERVISOR', 'STUDRES', 'HONVISIT', 'EXTERNAL_ADVISOR'], dtype=object)

In [109]:
for i in range(1,16):
    #print(pd.unique(unimelb["Home.Language."+str(i)]))
    print(pd.value_counts(unimelb["Home.Language."+str(i)]))

English    629
Other      169
Name: Home.Language.1, dtype: int64
English    223
Other       51
Name: Home.Language.2, dtype: int64
English    78
Other      19
Name: Home.Language.3, dtype: int64
English    29
Other       2
Name: Home.Language.4, dtype: int64
English    12
Other       2
Name: Home.Language.5, dtype: int64
English    4
Name: Home.Language.6, dtype: int64
English    2
Name: Home.Language.7, dtype: int64
Other      1
English    1
Name: Home.Language.8, dtype: int64
Other      1
English    1
Name: Home.Language.9, dtype: int64
Other      2
English    1
Name: Home.Language.10, dtype: int64
Series([], Name: Home.Language.11, dtype: int64)
Series([], Name: Home.Language.12, dtype: int64)
Series([], Name: Home.Language.13, dtype: int64)
Series([], Name: Home.Language.14, dtype: int64)
Series([], Name: Home.Language.15, dtype: int64)


In [110]:
for i in range(1,16):
    print(pd.unique(unimelb["Country.of.Birth."+str(i)]))

['Asia Pacific' 'Australia' 'Western Europe' nan 'North America'
 'Great Britain' 'New Zealand' 'The Americas' 'Eastern Europe'
 'South Africa' 'Middle East and Africa']
[nan 'Australia' 'Western Europe' 'New Zealand' 'Eastern Europe'
 'Asia Pacific' 'Great Britain' 'North America' 'South Africa'
 'Middle East and Africa' 'The Americas']
[nan 'Asia Pacific' 'Great Britain' 'Australia' 'North America'
 'South Africa' 'Eastern Europe' 'Western Europe' 'Middle East and Africa'
 'New Zealand' 'The Americas']
[nan 'Australia' 'North America' 'Western Europe' 'Eastern Europe'
 'Great Britain' 'Asia Pacific' 'South Africa' 'Middle East and Africa'
 'New Zealand' 'The Americas']
[nan 'Australia' 'Western Europe' 'North America' 'Asia Pacific'
 'South Africa' 'Great Britain' 'New Zealand' 'The Americas'
 'Eastern Europe']
[nan 'Australia' 'Great Britain' 'Western Europe' 'Asia Pacific'
 'North America' 'New Zealand']
[nan 'Australia' 'Western Europe' 'Great Britain' 'North America']
[nan 'Austr

In [117]:
unimelb.columns[26:41]

Index(['Person.ID.1', 'Role.1', 'Year.of.Birth.1', 'Country.of.Birth.1',
       'Home.Language.1', 'Dept.No..1', 'Faculty.No..1', 'With.PHD.1',
       'No..of.Years.in.Uni.at.Time.of.Grant.1',
       'Number.of.Successful.Grant.1', 'Number.of.Unsuccessful.Grant.1',
       'A..1', 'A.1', 'B.1', 'C.1'],
      dtype='object')

In [181]:
##Create a new dataframe with a column for the number of persons per team
tmp_df = pd.DataFrame(np.nan, index= unimelb['Grant.Application.ID'], columns=["num_of_pers"])

# add a column with each role from unimelb and then fill misisng values with 0
for i in range(1,16):
    tmp_df["Role."+ str(i)] = unimelb[["Role."+ str(i)]]
#replace all missing values by 0
tmp_df = tmp_df.fillna(0)
#replace all non-missing values by 1
tmp_df.replace('[^0]*', value = 1, regex = True, inplace = True)

#sum the number of roles in each line and put the sum into num_of_pers column
tmp_df["num_of_pers"] = tmp_df.sum(axis = 1)

#only keep the index ("grant ID") with the column "num_of_pers"
tmp_df = pd.DataFrame(tmp_df, columns=["num_of_pers"])

In [169]:
##Create a new dataframe with a column for the number of different countries per team
x = np.empty(len(unimelb))

tmp_df2 = pd.DataFrame(np.nan, index= unimelb['Grant.Application.ID'], columns=["CountCountries"])

for i in range(1,16):
    tmp_df2["Country.of.Birth."+ str(i)] = unimelb[["Country.of.Birth."+ str(i)]]

tmp_df2 = tmp_df2.fillna(0)
for i in range(len(unimelb)):
    x[i] = np.sum((pd.unique(tmp_df2.iloc[i,:]))!=0)
tmp_df2["CountCountries"] = x

#only keep the index ("grant ID") with the column "CountCountries"
tmp_df2 = pd.DataFrame(tmp_df2, columns=["CountCountries"])

In [173]:
##Create a new dataframe with a column for the average age per team
x = np.empty(len(unimelb))

tmp_df3 = pd.DataFrame(np.nan, index= unimelb['Grant.Application.ID'], columns=["AvAge"])

for i in range(1,16):
    tmp_df3["Year.of.Birth."+ str(i)] = unimelb[["Year.of.Birth."+ str(i)]]

for i in range(len(unimelb)):
    nan_ctr = tmp_df3.iloc[i,:].isnull().sum()
    x[i] = 2017 - (np.nansum(tmp_df3.iloc[i,:])/(16-nan_ctr))
tmp_df3["AvAge"] = x

#only keep the index ("grant ID") with the column "AvAge"
tmp_df3 = pd.DataFrame(tmp_df3, columns=["AvAge"])

/home/charles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [176]:
##Create a new dataframe with a column for the total number of years at university

x = np.empty(len(unimelb))

dict_years = {'Less than 0':0, '>=0 to 5':2.5, '>5 to 10':7.5, '>10 to 15':12.5,'more than 15':30}
tmp_df4 = pd.DataFrame(np.nan, index= unimelb['Grant.Application.ID'], columns=["NbYearsUni"])

for i in range(1,16):
    tmp_df4["No..of.Years.in.Uni.at.Time.of.Grant."+ str(i)] = unimelb[["No..of.Years.in.Uni.at.Time.of.Grant."+ str(i)]]

tmp_df4 = tmp_df4.replace(dict_years)
    
for i in range(len(df)):
    x[i] = np.nansum(tmp_df4.iloc[i,:])
tmp_df4["NbYearsUni"] = x

#only keep the index ("grant ID") with the column "NbYearsUni"
tmp_df4 = pd.DataFrame(tmp_df4, columns=["NbYearsUni"])

In [179]:
##Create a new dataframe with a column for the total number of PHD in each team

x = np.empty(len(unimelb))

tmp_df5 = pd.DataFrame(np.nan, index= unimelb['Grant.Application.ID'], columns=["Nb_Phd"])
for i in range(1,16):
    tmp_df5["With.PHD."+ str(i)] = unimelb[["With.PHD."+ str(i)]]

tmp_df5 = tmp_df5.fillna(0)
tmp_df5 = tmp_df5.replace('Yes ', 1)

for i in range(len(unimelb)):
    x[i] = np.sum(tmp_df5.iloc[i,:])
    
tmp_df5["Nb_Phd"] = x

#only keep the index ("grant ID") with the column "Nb_Phd"
tmp_df5 = pd.DataFrame(tmp_df5, columns=["Nb_Phd"])

In [64]:
#del(tmp_df11)
x = np.empty(len(df))
y = np.empty(len(df))
tmp_df11 = pd.DataFrame(np.nan, index=range(len(df)), columns=["num_of_depmts","main_depmt"])
#new_df = pd.DataFrame(np.nan, index=range(len(df)), columns=["num_of_pers"])

for i in range(1,16):
    #usefulcols.append(["Person.ID."+ str(i)])
    tmp_df11["Dept.No.."+ str(i)] = df[["Dept.No.."+ str(i)]]

tmp_df11= tmp_df11.fillna(0)
for i in range(len(df)):
    #tmp_df["num_of_pers"].iloc[i] = pd.unique(tmp_df.iloc[i,:])
    x[i] = np.sum(pd.unique(tmp_df11.iloc[i,:])>0)
    #y[i] = np.max(tmp_df11.iloc[i,:])
    
    if ( (pd.value_counts(tmp_df11.iloc[i,:])).index[0] == 0) and (len(pd.value_counts(tmp_df11.iloc[i,:]))>1 ) :
        y[i] = (pd.value_counts(tmp_df11.iloc[i,:])).index[1]
    else:
        y[i] = (pd.value_counts(tmp_df11.iloc[i,:])).index[0]
tmp_df11["num_of_depmts"] = x
tmp_df11["main_depmt"] = y
#new_df = tmp_df["num_of_pers"]
tmp_df11.head()

,num_of_depmts,main_depmt,Dept.No..1,Dept.No..2,Dept.No..3,Dept.No..4,Dept.No..5,Dept.No..6,Dept.No..7,Dept.No..8,Dept.No..9,Dept.No..10,Dept.No..11,Dept.No..12,Dept.No..13,Dept.No..14,Dept.No..15
0,1.0,3073.0,3073.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2538.0,2538.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2923.0,2923.0,2923.0,2498.0,2548.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,2678.0,2678.0,2678.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,2153.0,2153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#del(tmp_df12)
x = np.empty(len(df))
y = np.empty(len(df))
tmp_df12 = pd.DataFrame(np.nan, index=range(len(df)), columns=["num_of_facs","main_fac"])
#new_df = pd.DataFrame(np.nan, index=range(len(df)), columns=["num_of_pers"])

for i in range(1,16):
    tmp_df12["Faculty.No.."+ str(i)] = df[["Faculty.No.."+ str(i)]]

tmp_df12= tmp_df12.fillna(0)
for i in range(len(df)):
    #tmp_df["num_of_pers"].iloc[i] = pd.unique(tmp_df.iloc[i,:])
    x[i] = np.sum(pd.unique(tmp_df12.iloc[i,:])>0)
    #y[i] = np.max(tmp_df11.iloc[i,:])
    
    if ( (pd.value_counts(tmp_df12.iloc[i,:])).index[0] == 0) and (len(pd.value_counts(tmp_df12.iloc[i,:]))>1 ) :
        y[i] = (pd.value_counts(tmp_df12.iloc[i,:])).index[1]
    else:
        y[i] = (pd.value_counts(tmp_df12.iloc[i,:])).index[0]
tmp_df12["num_of_facs"] = x
tmp_df12["main_fac"] = y
#new_df = tmp_df["num_of_pers"]
tmp_df12.head(20)

,num_of_facs,main_fac,Faculty.No..1,Faculty.No..2,Faculty.No..3,Faculty.No..4,Faculty.No..5,Faculty.No..6,Faculty.No..7,Faculty.No..8,Faculty.No..9,Faculty.No..10,Faculty.No..11,Faculty.No..12,Faculty.No..13,Faculty.No..14,Faculty.No..15
0,1.0,31.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,25.0,25.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,31.0,31.0,31.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,31.0,31.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
#del(tmp_df14)
dict_role = {'CHIEF_INVESTIGATOR':7, 'DELEGATED_RESEARCHER':6, 'EXTERNAL_ADVISOR':5,
       'EXT_CHIEF_INVESTIGATOR':4, 'HONVISIT':3, 'PRINCIPAL_SUPERVISOR':2,
       'STUD_CHIEF_INVESTIGATOR':1, 'STUDRES':0}
x = np.empty(len(df))
y = np.empty(len(df))
tmp_df14 = pd.DataFrame(np.nan, index=range(len(df)), columns=["num_of_dif_Roles"])

for i in range(1,16):
    #usefulcols.append(["Person.ID."+ str(i)])
    tmp_df14["Role."+ str(i)] = df[["Role."+ str(i)]]

tmp_df14 = tmp_df14.replace(dict_role)

#for i in range(1,16):
#    print(pd.unique(tmp_df14["Role."+ str(i)]))

##tmp_df14= tmp_df14.fillna(0)
for i in range(len(df)):
    x[i] =   np.nansum(pd.unique(tmp_df14.iloc[i,:])>0)
    
tmp_df14["num_of_dif_Roles"] = x
tmp_df14.head(50)

/home/charles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater


,num_of_dif_Roles,Role.1,Role.2,Role.3,Role.4,Role.5,Role.6,Role.7,Role.8,Role.9,Role.10,Role.11,Role.12,Role.13,Role.14,Role.15
0,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,7.0,7.0,7.0,6.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2.0,7.0,4.0,4.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.0,7.0,7.0,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,7.0,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,7.0,7.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2.0,7.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df_newfeatures = pd.concat([tmp_df["num_of_pers"], tmp_df1["Nb_Phd"], tmp_df2["GrandSuc"], tmp_df3["GrandUnSuc"], 
                           tmp_df5["AApaper"], tmp_df6["Apaper"], tmp_df7["Bpaper"], tmp_df8["Cpaper"], 
                           tmp_df9["CountCountries"], tmp_df10["AvAge"], tmp_df11["num_of_depmts"],   
                           tmp_df11["main_depmt"], tmp_df12["num_of_facs"],tmp_df12["main_fac"], tmp_df13["NbYearsUni"], 
                            tmp_df14["num_of_dif_Roles"]], axis=1)
df_newfeatures.head()


,num_of_pers,Nb_Phd,GrandSuc,GrandUnSuc,AApaper,Apaper,Bpaper,Cpaper,CountCountries,AvAge,num_of_depmts,main_depmt,num_of_facs,main_fac,NbYearsUni,num_of_dif_Roles
0,1.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,1.0,52.00,1.0,3073.0,1.0,31.0,0.0,1.0
1,1.0,0.0,0.0,0.0,6.0,12.0,2.0,2.0,1.0,57.00,1.0,2538.0,1.0,25.0,30.0,1.0
2,4.0,0.0,0.0,0.0,7.0,20.0,20.0,7.0,2.0,65.75,3.0,2923.0,1.0,25.0,67.5,3.0
3,2.0,0.0,0.0,0.0,0.0,3.0,13.0,3.0,1.0,52.00,1.0,2678.0,1.0,25.0,10.0,4.0
4,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,52.00,1.0,2153.0,1.0,19.0,7.5,1.0


In [70]:
df_newfeatures.to_csv('grant_newfeatures.csv', index=False)

In [71]:
x = pd.read_csv("grant_newfeatures.csv")

In [72]:
x.shape

(8240, 16)

In [73]:
for i in range(1,16):
    #print(pd.value_counts(df["Role."+ str(i)], dropna=False))
    print(pd.unique(df["Role."+ str(i)]))

#tmp_df13.head()
#(pd.value_counts(tmp_df11.iloc[0,:])).index[2] 
#(pd.value_counts(tmp_df11.iloc[0,:])).values[1]
#pd.value_counts(tmp_df11.iloc[0,:])

#tmp_df11.head(20)
#np.nansum(tmp_df10.iloc[0,:]),tmp_df10.iloc[0,:].isnull().sum()
#np.nansum(tmp_df10.iloc[2,:]),tmp_df10.iloc[2,:].isnull().sum()
#np.nansum(pd.unique(tmp_df14.iloc[0,:])>0)
#np.nansum(pd.unique(tmp_df14.iloc[i,:])>0)

['CHIEF_INVESTIGATOR' 'PRINCIPAL_SUPERVISOR' 'DELEGATED_RESEARCHER'
 'EXT_CHIEF_INVESTIGATOR' 'HONVISIT']
[nan 'CHIEF_INVESTIGATOR' 'EXT_CHIEF_INVESTIGATOR'
 'STUD_CHIEF_INVESTIGATOR' 'DELEGATED_RESEARCHER' 'PRINCIPAL_SUPERVISOR'
 'STUDRES' 'HONVISIT' 'EXTERNAL_ADVISOR']
[nan 'CHIEF_INVESTIGATOR' 'EXT_CHIEF_INVESTIGATOR' 'DELEGATED_RESEARCHER'
 'STUD_CHIEF_INVESTIGATOR' 'PRINCIPAL_SUPERVISOR' 'STUDRES' 'HONVISIT'
 'EXTERNAL_ADVISOR']
[nan 'DELEGATED_RESEARCHER' 'EXT_CHIEF_INVESTIGATOR' 'CHIEF_INVESTIGATOR'
 'STUD_CHIEF_INVESTIGATOR' 'STUDRES' 'PRINCIPAL_SUPERVISOR' 'HONVISIT']
[nan 'EXT_CHIEF_INVESTIGATOR' 'CHIEF_INVESTIGATOR'
 'STUD_CHIEF_INVESTIGATOR' 'PRINCIPAL_SUPERVISOR' 'STUDRES'
 'DELEGATED_RESEARCHER']
[nan 'EXT_CHIEF_INVESTIGATOR' 'STUD_CHIEF_INVESTIGATOR'
 'CHIEF_INVESTIGATOR' 'DELEGATED_RESEARCHER' 'STUDRES'
 'PRINCIPAL_SUPERVISOR']
[nan 'EXT_CHIEF_INVESTIGATOR' 'CHIEF_INVESTIGATOR' 'STUDRES'
 'STUD_CHIEF_INVESTIGATOR']
[nan 'EXT_CHIEF_INVESTIGATOR' 'CHIEF_INVESTIGATOR' 'STU

In [74]:
tmp_df8["Cpaper"]

0        0.0
1        2.0
2        7.0
3        3.0
4        0.0
5        6.0
6        4.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       1.0
13       0.0
14       2.0
15       2.0
16       0.0
17       2.0
18      32.0
19       0.0
20       0.0
21       0.0
22       2.0
23       1.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       2.0
        ... 
8210     3.0
8211     0.0
8212     0.0
8213     0.0
8214    37.0
8215     0.0
8216     1.0
8217     3.0
8218     6.0
8219    13.0
8220     0.0
8221     0.0
8222     2.0
8223     0.0
8224    19.0
8225     0.0
8226     5.0
8227     1.0
8228     0.0
8229     2.0
8230     8.0
8231     0.0
8232     8.0
8233     5.0
8234     0.0
8235     2.0
8236     0.0
8237    22.0
8238     7.0
8239     2.0
Name: Cpaper, Length: 8240, dtype: float64

In [75]:
for i in range(1,16):
    a = (unimelb["Person.ID."+str(i)].isnull() & ~(unimelb["Role."+str(i)].isnull()))
    a = a.sum()
    b = a.sum()*100/unimelb.shape[0]
    print(a, b)   # persons ID na but Role specified

425 4.88056959118
1413 16.226458429
1057 12.1382636656
732 8.40606338999
412 4.73128158016
240 2.75608635737
58 0.666054203032
41 0.470831419384
26 0.298576022049
21 0.24115755627
14 0.16077170418
9 0.103353238401
8 0.0918695452458
0 0.0
6 0.0689021589343


In [76]:
for i in range(1,16):
    a = unimelb["Role."+str(i)].value_counts()
    print(a, " ", sep="\n\n") 

CHIEF_INVESTIGATOR         7741
EXT_CHIEF_INVESTIGATOR      413
PRINCIPAL_SUPERVISOR        332
DELEGATED_RESEARCHER        101
STUD_CHIEF_INVESTIGATOR      11
HONVISIT                     11
EXTERNAL_ADVISOR              1
Name: Role.1, dtype: int64

 
CHIEF_INVESTIGATOR         2705
EXT_CHIEF_INVESTIGATOR      958
STUD_CHIEF_INVESTIGATOR     375
DELEGATED_RESEARCHER        140
PRINCIPAL_SUPERVISOR        130
STUDRES                      77
HONVISIT                     14
EXTERNAL_ADVISOR              3
Name: Role.2, dtype: int64

 
CHIEF_INVESTIGATOR         1050
EXT_CHIEF_INVESTIGATOR      890
STUD_CHIEF_INVESTIGATOR     124
PRINCIPAL_SUPERVISOR         42
STUDRES                      42
DELEGATED_RESEARCHER         40
HONVISIT                      2
EXTERNAL_ADVISOR              1
Name: Role.3, dtype: int64

 
EXT_CHIEF_INVESTIGATOR     659
CHIEF_INVESTIGATOR         383
STUD_CHIEF_INVESTIGATOR     54
DELEGATED_RESEARCHER        20
PRINCIPAL_SUPERVISOR        19
STUDRES            

In [77]:
for i in range(1,16):
    a = unimelb["With.PHD."+str(i)].value_counts()
    print(a, " ", sep="\n\n")  

Yes     5113
Name: With.PHD.1, dtype: int64

 
Yes     1748
Name: With.PHD.2, dtype: int64

 
Yes     653
Name: With.PHD.3, dtype: int64

 
Yes     275
Name: With.PHD.4, dtype: int64

 
Yes     100
Name: With.PHD.5, dtype: int64

 
Yes     44
Name: With.PHD.6, dtype: int64

 
Yes     25
Name: With.PHD.7, dtype: int64

 
Yes     8
Name: With.PHD.8, dtype: int64

 
Yes     7
Name: With.PHD.9, dtype: int64

 
Yes     5
Name: With.PHD.10, dtype: int64

 
Yes     3
Name: With.PHD.11, dtype: int64

 
Yes     5
Name: With.PHD.12, dtype: int64

 
Yes     3
Name: With.PHD.13, dtype: int64

 
Series([], Name: With.PHD.14, dtype: int64)

 
Series([], Name: With.PHD.15, dtype: int64)

 


In [78]:
for i in range(1,16):
    a = (unimelb["Person.ID."+str(i)].isnull())
    a = a.sum()
    b = a.sum()*100/unimelb.shape[0]
for i in range(1,16):
    a = (unimelb["Person.ID."+str(i)].isnull() & ~(unimelb["Role."+str(i)].isnull()))
    a = a.sum()
    b = a.sum()*100/unimelb.shape[0]
    print(a, b)   # persons ID na but Role specified    print(a, b)   # persons ID na but Role specified

425 4.88056959118
1413 16.226458429
1057 12.1382636656
732 8.40606338999
412 4.73128158016
240 2.75608635737
58 0.666054203032
41 0.470831419384
26 0.298576022049
21 0.24115755627
14 0.16077170418
9 0.103353238401
8 0.0918695452458
0 0.0
6 0.0689021589343


In [79]:
for i in range(1,16):
    a = ( unimelb["Person.ID."+str(i)].isnull() & (unimelb["Role."+str(i)].isnull()) & unimelb["Home.Language."+str(i)].isnull() &\
        (unimelb["Faculty.No.."+str(i)].isnull()) & (unimelb["With.PHD."+str(i)].isnull()) & (unimelb["No..of.Years.in.Uni.at.Time.of.Grant."+str(i)].isnull()) &\
        (unimelb["Number.of.Successful.Grant."+str(i)].isnull()) & (unimelb["A.."+str(i)].isnull()) & unimelb["A."+str(i)].isnull())
    a = a.sum()
    b = a.sum()*100/unimelb.shape[0]
    print(a, b)   # persons ID na but Role specified
    

98 1.12540192926
4306 49.4487827285
6517 74.8392282958
7552 86.724850712
8140 93.4772622876
8398 96.4400551217
8614 98.9205328434
8655 99.3913642627
8674 99.6095544327
8681 99.6899402848
8689 99.78180983
8694 99.8392282958
8697 99.8736793753
8708 100.0
8700 99.9081304548


In [80]:
for i in range(1,7):
    a = (unimelb["Person.ID."+str(i)].isnull())
    a = a.sum()
    print(a)   # persons ID na but Role specified
for i in range(1,7):
    a = (unimelb["Person.ID."+str(i)].describe())
    print(a)   # persons ID na but Role specified

523
5719
7574
8284
8552
8638
count    8.185000e+03
mean     1.657378e+05
std      2.474406e+05
min      3.570000e+02
25%      2.850200e+04
50%      7.172700e+04
75%      1.145520e+05
max      1.162272e+06
Name: Person.ID.1, dtype: float64
count    2.989000e+03
mean     1.507789e+05
std      2.257829e+05
min      3.570000e+02
25%      2.860700e+04
50%      7.203700e+04
75%      1.104370e+05
max      1.092302e+06
Name: Person.ID.2, dtype: float64
count      1134.000000
mean     143339.852734
std      207758.710269
min         462.000000
25%       33167.000000
50%       73087.000000
75%       99459.500000
max      996927.000000
Name: Person.ID.3, dtype: float64
count       424.000000
mean     118938.497642
std      186526.767412
min         362.000000
25%       32518.250000
50%       68602.000000
75%       81772.000000
max      982227.000000
Name: Person.ID.4, dtype: float64
count    1.560000e+02
mean     1.483559e+05
std      2.372370e+05
min      5.370000e+02
25%      3.285700e+04
50%  

In [ ]:
df.shape, unimelb.shape

Person.ID.1
Role.1
Year.of.Birth.1
Country.of.Birth.1
Home.Language.1
Dept.No..1
Faculty.No..1
With.PHD.1
No..of.Years.in.Uni.at.Time.of.Grant.1
Number.of.Successful.Grant.1
Number.of.Unsuccessful.Grant.1
A..1
A.1
B.1
C.1

In [ ]:
cor1 = df[["Grant.Status", "Role.1"]]
cat1 = pd.Categorical(df["Role.1"])
#pd.unique(cat1.labels)
cor1.loc[:,"Role.1"] = cat1.labels
#cat1.labels
cor1.corr()
#np.corrcoef([df["Grant.Status"],df["Role.1"]])


In [ ]:
dict_cat = {'CHIEF_INVESTIGATOR':7, 'DELEGATED_RESEARCHER':6, 'EXTERNAL_ADVISOR':5,
       'EXT_CHIEF_INVESTIGATOR':4, 'HONVISIT':3, 'PRINCIPAL_SUPERVISOR':2,
       'STUD_CHIEF_INVESTIGATOR':1}

cor_mean_Role = 0
cor = 0
for i in range(1,2):
    cor_t = unimelb[["Grant.Status", "Role."+str(i)]]
    cat1 = cor_t
    #cat1 = cat1.rename(columns={'Role.1': 'Role.'+str(i)})
    cat1 = cat1.replace({"Role."+str(i): dict_cat})
    #print(cat1)
    cor_t["Role."+str(i)] = cat1
    cor = cor_t.corr()
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)

In [ ]:
dict_cat = {'CHIEF_INVESTIGATOR':7, 'DELEGATED_RESEARCHER':6, 'EXTERNAL_ADVISOR':5,
       'EXT_CHIEF_INVESTIGATOR':4, 'HONVISIT':3, 'PRINCIPAL_SUPERVISOR':2,
       'STUD_CHIEF_INVESTIGATOR':1}

cor_mean_Role = 0
cor = 0
for i in range(1,9):
    cor_t = unimelb[["Grant.Status", "Role."+str(i)]]
    #cor_t["Role."+str(i)] = cor_t.replace({"Role."+str(i): dict_cat})
    cor_t.rename(columns={'gdp':'log(gdp)'}, inplace=True)
    cor_t = cor_t.replace({"Role."+str(i): dict_cat})
    print(cor_t)
    cor = cor_t.corr()
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)

In [ ]:
cor_mean_Role = 0
cor = 0
for i in range(1,16):
    cor_t = unimelb[["Grant.Status", "Number.of.Successful.Grant."+str(i)]]
    cat1 = pd.Categorical(cor_t["Number.of.Successful.Grant."+str(i)])
    cor_t["Number.of.Successful.Grant."+str(i)] = cat1.labels
    cor = cor_t.corr()
    #print(cor_t.corr())
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)


In [ ]:
cor_mean_Role = 0
cor = 0
for i in range(1,16):
    cor_t = unimelb[["Grant.Status", "A.."+str(i)]]
    cat1 = pd.Categorical(cor_t["A.."+str(i)])
    cor_t["A.."+str(i)] = cat1.labels
    cor = cor_t.corr()
    #print(cor_t.corr())
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)


In [ ]:
cor_mean_Role = 0
cor = 0
for i in range(1,16):
    cor_t = unimelb[["Grant.Status", "Number.of.Unsuccessful.Grant."+str(i)]]
    cat1 = pd.Categorical(cor_t["Number.of.Unsuccessful.Grant."+str(i)])
    cor_t["Number.of.Unsuccessful.Grant."+str(i)] = cat1.labels
    cor = cor_t.corr()
    #print(cor_t.corr())
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)

In [ ]:
cor_mean_Role = 0
cor = 0
for i in range(1,16):
    cor_t = unimelb[["Grant.Status", "C."+str(i)]]
    cat1 = pd.Categorical(cor_t["C."+str(i)])
    cor_t["C."+str(i)] = cat1.labels
    cor = cor_t.corr()
    #print(cor_t.corr())
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)


In [ ]:
cor_mean_Role = 0
cor = 0
for i in range(1,16):
    cor_t = unimelb[["Grant.Status", "With.PHD."+str(i)]]
    cat1 = pd.Categorical(cor_t["With.PHD."+str(i)])
    cor_t["With.PHD."+str(i)] = cat1.labels
    cor = cor_t.corr()
    #print(cor_t.corr())
    cor_mean_Role = (cor_mean_Role + cor.iloc[0,1]) / 15
    print(cor_mean_Role)

In [ ]:
dict_cat2 = {7:'CHIEF_INVESTIGATOR', 6:'DELEGATED_RESEARCHER', 5:'EXTERNAL_ADVISOR',
       4:'EXT_CHIEF_INVESTIGATOR', 3:'HONVISIT', 2:'PRINCIPAL_SUPERVISOR',
       1:'STUD_CHIEF_INVESTIGATOR'}

cor_t = unimelb[["Grant.Status", "Role.1"]]
cat1 = pd.Categorical(cor_t["Role.1"], ordered=True)
cat1.categories

In [ ]:
t = cor_t
t.replace({"Role.1": dict_cat})
#t["Role.1"].update(pd.Series(dict_cat2))